In [15]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
import os

In [5]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [6]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# llm = OllamaFunctions(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", format="json", temperature=0)
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_FAISS_INDEX = "MY_FAISS_INDEX"
vectorstore1 = FAISS.load_local(MY_FAISS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사도 높은 5문장 추출

/Users/kwon0koang/Library/Caches/pypoetry/virtualenvs/test-langchain-PJ4Uqdqm-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/kwon0koang/Library/Caches/pypoetry/virtualenvs/test-langchain-PJ4Uqdqm-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.pydantic_v1 import BaseModel, Field

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="web_search",
    description="엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다",
)

tools = [retriever_tool1, retriever_tool2]
tools

[Tool(name='web_search', description='엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12e246660>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2bb6948f0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x12e246b60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2bb6948f0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description='생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다', args_schema=<class 'langchain_core.too

In [9]:
# llm_with_tools = llm.bind_tools(tools=tools)
# response = llm_with_tools.invoke("What is 3*12=? Also, What is 11+43?")
# print(f"response: {response}")
# print(f"tool_calls: {response.tool_calls}")

In [10]:
prompt_for_select_tool = ChatPromptTemplate.from_messages([
    ("system", """
Select one ‘tool’ to indicate which tool you would use to answer the ‘question’ correctly.
Say only the ‘name’ of the ‘tool’ without saying anything else.

<tools>
{tools}
</tools>

<question>
{question}
</question>

# answer :
"""
    )
])

def get_tools(query):
    tool_info = [(tool.name, tool.description) for tool in tools]
    print(f"get_tools / {tool_info}") # [('web_search', '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다'), ('pdf_search', '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다')]
    return str(tool_info)

chain_for_select_tool = (
    {"tools": get_tools, "question": RunnablePassthrough()}
    | prompt_for_select_tool 
    | llm
    | StrOutputParser()
    )

In [11]:
query = "라마3 성능은?"
selected_tool = chain_for_select_tool.invoke(query)
selected_tool

get_tools / [('web_search', '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다'), ('pdf_search', '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다')]


'web_search'

In [12]:
def get_retriever_by_tool_name(name) -> VectorStoreRetriever:
    print(f"get_retriever_by_tool_name / name: {name}")
    for tool in tools:
        if tool.name == name:
            # print(tool.func) # functools.partial(<function _get_relevant_documents at 0x1487dd6c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317e52ea0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')
            return tool.func.keywords['retriever']
    return None

retriever = get_retriever_by_tool_name(selected_tool)
retriever.invoke(query)

get_retriever_by_tool_name / name: web_search


[Document(page_content="라마 3 벤치마크 결과 (사진=메타)\n\n\n라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에는 강하지만, 70B의 경우 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 떨어졌다.\xa0\n특히 인간 선호도에서 경쟁 모델을 앞서는 것으로 알려졌다.\n조언 요청, 브레인스토밍, 분류, 비공개 질문 답변, 코딩, 창의적인 글쓰기, 추출, 공개 질문 답변, 추론, 재작성 및 요약 등 12가지 주요 사용 사례를 포함한 1800개\xa0프롬프트\xa0구축\xa0데이터셋에 대한 인간 평가에서 오픈AI의 'GPT-3.5', 미스트랄 7B, 클로드 3 소네트보다 높게 평가됐다.\n\n\n라마 3 인간 평가 결과 (사진=메타)", metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='라마 3 인간 평가 결과 (사진=메타)\n\n\n허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드\xa01위에 오르며 역대 가장 빠른 1위 달성 기록을 세웠다.\n또 이전 라마 1과 2를 기반으로 3만개 이상의 새로운 모델이 출시됐으며, 라마 2 모델은 1700억번 다운로드됐다는 통계치도 공개해 눈길을 모았다.\xa0\n다만 라마 3는 완전한 오픈 소스가 아니다.\xa0연구용 및 상업용으로 모두 사용할 수 있지만, 개발자가 다른 생성 모델을 훈련하기 위해 모델을 사용하는 것을 금지한다.\n\n\n메타 AI (사진=메타)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='특히 15조개 이상의 토큰을 동원, 학습량이 라마 2 대비 7배 이상 많으며 코드량은 4배 더 많다. 다만 데이터셋은 공

In [16]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 유능한 업무 보조자야.
다음 context를 사용해서 question에 대한 답을 말해줘.
정답을 모르면 모른다고만 해.

<question>
{question}
</question>

<context>
{context}
</context>

# answer :
"""
    ),
])

retrieved_docs = []
def get_page_contents_with_metadata(docs) -> str: 
    global retrieved_docs
    retrieved_docs = docs
    
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc.page_content}\n### 출처: {doc.metadata['source']}"
    return result

def get_retrieved_docs(query) -> str:
    selected_tool = chain_for_select_tool.invoke(query)
    retriever = get_retriever_by_tool_name(selected_tool)
    docs = retriever.invoke(query)
    return get_page_contents_with_metadata(docs)

def get_metadata_sources(docs) -> str: 
    sources = set()
    for doc in docs:
        source = doc.metadata['source']
        is_pdf = source.endswith('.pdf')
        if (is_pdf):
            file_path = doc.metadata['source']
            file_name = os.path.basename(file_path)
            source = f"{file_name} ({doc.metadata['page']}페이지)"
        sources.add(source)
    return "\n".join(sources)

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message and add source."""
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

agent_chain = (
    {"context": get_retrieved_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parse
)

# final_chain.invoke("퍼플렉시티가 투자받은 금액?") # web_search / '퍼플렉시티는 투자받은 금액이 약 6300만 달러(약 860억 원)이며, 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가받았습니다.'
agent_chain.invoke("생성형 AI 도입에 따른 규제 연구 책임자는?") # pdf_search / '해당 문서에 따르면, 생성형 AI 도입과 관련된 규제 연구를 담당하는 연구 책임자는 김주희 교수님입니다.'

get_tools / [('web_search', '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다'), ('pdf_search', '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다')]
get_retriever_by_tool_name / name: pdf_search


'해당 문서에 따르면, 생성형 AI 도입과 관련된 규제 연구를 담당하는 연구 책임자는 김주희 교수님입니다.\n\n[출처]\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (6페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (20페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (21페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (1페이지)'